In [11]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [12]:
user_credential = r'\Users\dinhhoang.nguyen.CONCENTRIX\OneDrive - Concentrix Corporation\WFM-internal'
kpi = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\kpi.json'))
kpi['Date'] = kpi['Date'].dt.date

time_update = dt(year=2023, month=1, day=1)

In [13]:
# Get prod target
requirement = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\requirement.json'))[['LOB', 'Date', 'Daily Requirement']]
requirement['Date'] = requirement['Date'].dt.date
requirement = requirement.rename(columns={'Daily Requirement': 'Productive Target'})
# add prod target to kpi
kpi = pd.merge(kpi, requirement, on=['LOB', 'Date'], how='left')
kpi['Productive Target'] = kpi['Productive Target'].fillna(0)

In [14]:
# Here we calculate IO
daily_tracking = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\dailytracking_full.json'))[['Session Date', 'EID', 'Break Time', 'Other codes', 'Staffed Hours', 'Delivery hours']]
daily_tracking['Session Date'] = pd.to_datetime(daily_tracking['Session Date'], format='mixed').dt.date
# Merge kpi with daily tracking
kpi = pd.merge(kpi, daily_tracking, left_on=['Date', 'Emp ID'], right_on=['Session Date', 'EID'], how='left')
# Drop 'Session Date', 'EID'
kpi = kpi.drop(columns={'Session Date', 'EID'})

kpi[['Break Time', 'Other codes', 'Staffed Hours', 'Delivery hours']] = kpi[['Break Time', 'Other codes', 'Staffed Hours', 'Delivery hours']]*3600
kpi[['Break Time', 'Other codes', 'Staffed Hours', 'Delivery hours']] = kpi[['Break Time', 'Other codes', 'Staffed Hours', 'Delivery hours']].fillna('0')

# # Convert Break Time to seconds
# kpi[['hrs', 'mins', 'secs']] =  kpi['Break Time'].str.split(':', expand=True)
# kpi['Break Time'] = kpi['hrs'].astype('int64')*3600 + kpi['mins'].astype('int64')*60 + kpi['secs'].astype('int64')
# # Convert Other codes to seconds
# kpi[['hrs', 'mins', 'secs']] =  kpi['Other codes'].str.split(':', expand=True)
# kpi['Other codes'] = kpi['hrs'].astype('int64')*3600 + kpi['mins'].astype('int64')*60 + kpi['secs'].astype('int64')
# # Convert Staffed Hours to seconds
# kpi[['hrs', 'mins', 'secs']] =  kpi['Staffed Hours'].str.split(':', expand=True)
# kpi['Staffed Hours'] = kpi['hrs'].astype('int64')*3600 + kpi['mins'].astype('int64')*60 + kpi['secs'].astype('int64')
# # Convert Delivery hours to seconds
# kpi[['hrs', 'mins', 'secs']] =  kpi['Delivery hours'].str.split(':', expand=True)
# kpi['Delivery hours'] = kpi['hrs'].astype('int64')*3600 + kpi['mins'].astype('int64')*60 + kpi['secs'].astype('int64')

# kpi = kpi.drop(columns={'hrs', 'mins', 'secs'})

In [15]:
# Export to csv
kpi.loc[(pd.to_datetime(kpi['Date'], format='mixed')>time_update)].to_csv(os.path.join(r'C:', user_credential, r'DB\filecsv\aidashboard.csv'), index=False)

: 